# **Programming Assessment \#4**

Names: \<please supply your names\>

More information on the assessment is found in our Canvas course.

# **Load Data**

*While you don't have to separate your code into blocks, it might be easier if you separated loading your data from actually implementation of your code. Consider placing all loading of data into the code block below.*

In [102]:
# The code is defining a function called `make_p` that takes a DataFrame `df` as input.
# lang must contain a column 'count' that contains number types
#load error data
import pandas as pd
def make_p(lang):
    total = lang['count'].sum()
    inv = 1/total
    df['P'] = err['count']*inv
    return df
x=''

with open("count_1edit.txt", "r") as f:
    x = f.read().splitlines()
gen = (dat.split('\t') for dat in x)
data = [(rec[0],int(rec[1]))for rec in gen]
err = pd.DataFrame.from_records(data, columns=['error', 'count'])
err = make_p(err)
err = err.set_index(['error'])


       count         P
error                 
e|i      917  0.023469
a|e      856  0.021908
i|e      771  0.019732
e|a      749  0.019169
a|i      559  0.014307
...      ...       ...
 |c        1  0.000026
 |a        1  0.000026
 |'        1  0.000026
 w|        1  0.000026
 t|t       1  0.000026

[1587 rows x 2 columns]


In [72]:
#loading corpus files
import nltk
from nltk.corpus import gutenberg
nltk.download('gutenburg')
# nltk.corpus.gutenberg.fileids()

[nltk_data] Error loading gutenburg: Package 'gutenburg' not found in
[nltk_data]     index


False

In [104]:
from collections import Counter

import string
print("Extracting all documents from NLTK's Project Gutenberg Collection...")
all_words = Counter()
for filename in nltk.corpus.gutenberg.fileids():
  words = [word.lower() for word in nltk.corpus.gutenberg.words(filename) if word not in string.punctuation]
  all_words.update(words)
lang = pd.DataFrame.from_dict(all_words, orient='index').reset_index()


Extracting all documents from NLTK's Project Gutenberg Collection...


In [105]:
lang = lang.rename(columns={'index':'word', 0:'count'})
lang = make_p(lang)
lang = lang.set_index(['word'])

# **Noisy Channel Model Implementation**

*Again, you don't have to follow this directly, but consider placing your implementation of the model in the code block below.*

In [75]:
import numpy as np

def find_err(str_1, str_2):
    rows = len(str_1) + 1
    cols = len(str_2) + 1
    dMatrix = np.zeros((rows, cols), dtype=int)
    dMatrix[0]= [*range(0, cols)]
    for i in range(1, rows):
        dMatrix[i][0] = i
            
        
    for col in range(1, cols):
        for row in range(1, rows):
            if str_1[row - 1] == str_2[col - 1]:
               cost = 0
            else:
               cost = 1
            if str_1[row - 1] == str_2[col] and str_1[row] == str_2[col-1]:
                # transpose
                pass
            dMatrix[row][col] = min(dMatrix[row - 1][col] + 1, dMatrix[row][col - 1] + 1, dMatrix[row-1][col-1] + cost)

    
    return dMatrix[row][col]


In [76]:
# Shawn's trash
# from collections import Counter
# import numpy as np
# from re import finditer,compile
# from editdistpy import damerau_osa as ld
# import pandas as pd
# import nltk
# from nltk import FreqDist as f_dist
# from nltk.corpus import gutenberg as cor_g
# import re 
# class L_Model:
# 	def __init__(self):
# 		#dowload corpus
# 		nltk.download('gutenburg')
# 		nltk.corpus.gutenberg.fileids()
# 		corpus = Counter()
# 		for filename in nltk.corpus.gutenberg.fileids():
# 			words = [word.lower() for word in nltk.corpus.gutenberg.words(filename)]
# 			corpus.update(words)
		
# 		#corpus to df
# 		df = pd.DataFrame.from_dict(corpus, orient='index').reset_index()
# 		df = df.rename(columns={'index':'word', 0:'count'})


		
# class E_Model:
	
# 	def __init__(self,data:list[tuple[str,int]]):
# 		df = pd.DataFrame(data, columns=['error', 'count'])
# 		total = sum(df['count'])
# 		inv = 1/total
# 		df['p'] = df['count']*inv
# 		self.df = df
	
# 	#use regex to parse norvig's 1edit err
# 	letterPattern = r"[a-z|]+"
# 	numberPattern = r"\d+"

# 	file1 = open('count_1edit.txt', 'r')
# 	error_list = []
# 	counts = []
		
# 	for line in file1:
# 		letterMatch = re.findall(letterPattern, line)
# 		numberMatch = re.findall(numberPattern, line)
# 		error = letterMatch.pop(0)
# 		count = numberMatch.pop(0)
# 		error_list.append(error)
# 		counts.append(count)

# class Finder:
# 	def __init__(self):

# 		self.model = L_Model()
	
# 		# if not isinstance(err,pd.DataFrame):
# 		# 	err = pd.DataFrame(err)
# 		# self.err = err
			
# 	def getD(self,tok:str):
# 		candidates = []
# 		d = 1
# 		MAX = 90 #magic number set to the longest possible edit distance
# 		while not candidates and d <= MAX:
# 			candidates = [w for w in self.model.index if ld.distance(tok,w,d) > -1]
# 			d+=1
# 		return candidates
	
# 	def getCandidates(self,tok:str):
# 		candidates = [tok]
# 		if tok not in self.model:
# 			candidates = self.getD(tok)
# 			p_c =self.model[candidates]
# 			candidates = self.refine(p_c)
# 		return candidates

In [80]:
%pip install editdistpy
from editdistpy import damerau_osa as ld

def candidates(input, all_words):
      # 1 edit distance away from word
      if input in all_words: return [input]
      d = 1
      candidate_list = [w for w in all_words if ld.distance(input,w,d) > -1]
      # for word in all_words:
      #       x = min_edit_distance(input, word)
      #       if x == 1:
      #             candidate_list.append(word)
      return candidate_list

def spell_correct(candidate_list):
      n=1
      
      probable = df.loc[candidate_list].sort_values(['P'], ascending=False).head(n)
      # maxI = 0
      # for i in range(len(candidate_list)):
      #       x = P(candidate_list[i], all_words)
      #       if x > maxX:
      #             maxX = x
      #             maxI = i
      # TODO: Add the error model. Kyle, can you please make an algo that finds the first error and query the probability?
      return list(probable.index)[0], probable['P']

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [78]:
# def calculate_edit_type_and_edit(input_word, candidate_word):
#     if len(input_word) == len(candidate_word):
#         # Check for substitution
#         diff_count = 0
#         edit_type = "sub"
#         edit = ""
#         for i in range(len(input_word)):
#             if input_word[i] != candidate_word[i]:
#                 diff_count += 1
#                 edit += candidate_word[i]
#         if diff_count == 1:
#             return edit_type, edit

#     elif len(input_word) + 1 == len(candidate_word):
#         # Check for insertion
#         edit_type = "ins"
#         edit = ""
#         i, j = 0, 0
#         while i < len(input_word) and j < len(candidate_word):
#             if input_word[i] != candidate_word[j]:
#                 edit += candidate_word[j]
#                 j += 1
#             else:
#                 i += 1
#                 j += 1
#         if j == len(candidate_word):
#             return edit_type, edit

#     elif len(input_word) - 1 == len(candidate_word):
#         # Check for deletion
#         edit_type = "del"
#         edit = ""
#         i, j = 0, 0
#         while i < len(input_word) and j < len(candidate_word):
#             if input_word[i] != candidate_word[j]:
#                 edit += input_word[i]
#                 i += 1
#             else:
#                 i += 1
#                 j += 1
#         if i == len(input_word):
#             return edit_type, edit

#     # Check for transposition
#     if len(input_word) == len(candidate_word) and input_word != candidate_word:
#         for i in range(len(input_word) - 1):
#             if input_word[i] == candidate_word[i + 1] and input_word[i + 1] == candidate_word[i]:
#                 edit_type = "trans"
#                 edit = input_word[i:i+2]
#                 return edit_type, edit

#     return None, None  # No valid edit type found

# # Get user input
# #user_input = input("Enter a word for spell correction: ")

# corpus = "This is a sample text for the spell correction model. Modeler is working fine."
# model = Modeler(corpus)

# print(model.model)

# finder = Finder(model.model)

# token = "food"
# candidates = finder.getCandidates(token)

# print(candidates)

# results = []
# valid_edit_types = ["ins", "trans", "sub" ,"del", ]
# for candidate in candidates:
#     candidate_word, probability = candidate[0], candidate[1]
#     edit_type, edit = calculate_edit_type_and_edit(token, candidate_word)
#     print("\n")
#     print(edit_type,edit)
#     print("\n")
#     if edit_type in valid_edit_types:
#         result = {
#             "word": token,
#             "candidate": candidate_word,
#             "edit_type": edit_type,
#             "edit": edit,
#             "P(c)": probability,
#             "P(w|c)": model.model[candidate_word],
#             "P(c) x P(w|c)": probability * model.model[candidate_word]
#         }
#         results.append(result)
#     else:
#         result = "broken"
#         results.append(result)

# df = pd.DataFrame(results)

# print(df)


In [81]:
#main function
test_input = "mohter"
c = candidates(test_input, all_words)
spell_correct(c)

('mother',
 word
 mother    0.000545
 Name: P, dtype: float64)